In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import os

%matplotlib inline

In [ ]:
parent_dir = os.path.split(os.getcwd())[0]

In [ ]:
zones = gpd.read_file(parent_dir + '\\Data\\New\\lms_zone_du_new.shp')
ovin_tb = pd.read_csv(parent_dir + '\\Data\\New\\lms_zone_ovin_travel_behaviour.csv', index_col=0)
lms_tb = pd.read_csv(parent_dir + '\\Data\\New\\lms_zone_lms_modal_split.csv', index_col=0)

In [ ]:
zones['Count'] = np.zeros(len(zones))

In [ ]:
zones.loc[ovin_tb.Count_o > 20, 'Count'] = 1 # Mark all zones with 20 or less data points

## Functions for plotting and selecting data

In [ ]:
def select_zones(name_list, area_type='GEM_NAAM', du=None):
    """Select the zones to use
    
    Area types:
    - "GEM_NAAM"
    - "PROV"
    """

    zone_df = zones[zones[area_type].isin(name_list)]

    if du is not None:
        zone_df = zone_df[zone_df['deg_urba'] == du]

    return zone_df

In [ ]:
def select_ovin_lms(zone, direction='o'):
    """
    Select the zones in the ovin and lms dataframe

    - 'o' is origin
    - 'd' is destination
    """

    # Make arrays of all zones_IDs in that zone
    zone_arr = np.array(zone['ZONE_ID'])

    ovin_df = ovin_tb[ovin_tb.ZONE_ID.isin(zone_arr)]
    lms_df = lms_tb[lms_tb.ZONE_ID.isin(zone_arr)]

    if direction == 'o':
        ovin_df = ovin_df.iloc[:, np.r_[0:7]]
        lms_df = lms_df.iloc[:, np.r_[0:8]]

    elif direction == 'd':
        ovin_df = ovin_df.iloc[:, np.r_[0, 7:13]]
        lms_df = lms_df.iloc[:, np.r_[0, 8:15]]
    
    else:
        print('No valid direction')
    
    return ovin_df, lms_df

In [ ]:
pd.options.mode.chained_assignment = None

In [ ]:
def select_modes(zone, ovin_df, lms_df, modes='c-btm-a'):
    """
    Create zone dataframe with the right modes
    """

    if modes == 'c-btm-a':

        zone.loc[:, 'car_ovin'] = ovin_df.iloc[:, 1:3].sum(axis=1)
        zone.loc[:, 'car_lms'] = lms_df.iloc[:, 1:3].sum(axis=1)
        zone.loc[:, 'car_diff'] = ovin_df.iloc[:, 1:3].sum(axis=1) - lms_df.iloc[:, 1:3].sum(axis=1)

        zone.loc[:, 'btm_ovin'] = ovin_df.iloc[:, 3:5].sum(axis=1)
        zone.loc[:, 'btm_lms'] = lms_df.iloc[:, 3:6].sum(axis=1)
        zone.loc[:, 'btm_diff'] = ovin_df.iloc[:, 3:5].sum(axis=1) - lms_df.iloc[:, 3:6].sum(axis=1)

        zone.loc[:, 'active_ovin'] = ovin_df.iloc[:, 5:].sum(axis=1)
        zone.loc[:, 'active_lms'] = lms_df.iloc[:, 6:].sum(axis=1)
        zone.loc[:, 'active_diff'] = ovin_df.iloc[:, 5:].sum(axis=1) - lms_df.iloc[:, 6:].sum(axis=1)

    elif modes == 'all':

        zone.loc[:, 'card_ovin'] = ovin_df.iloc[:, 1:2].sum(axis=1)
        zone.loc[:, 'card_lms'] = lms_df.iloc[:, 1:2].sum(axis=1)
        zone.loc[:, 'card_diff'] = (- ovin_df.iloc[:, 1:2].sum(axis=1) + lms_df.iloc[:, 1:2].sum(axis=1)) / ovin_df.iloc[:, 1:2].sum(axis=1) * 100
        # zone.loc[:, 'card_diff'] = (ovin_df.iloc[:, 1:2].sum(axis=1) - lms_df.iloc[:, 1:2].sum(axis=1))

        zone.loc[:, 'carp_ovin'] = ovin_df.iloc[:, 2:3].sum(axis=1)
        zone.loc[:, 'carp_lms'] = lms_df.iloc[:, 2:3].sum(axis=1)
        zone.loc[:, 'carp_diff'] = (- ovin_df.iloc[:, 2:3].sum(axis=1) + lms_df.iloc[:, 2:3].sum(axis=1)) / ovin_df.iloc[:, 2:3].sum(axis=1) * 100
        # zone.loc[:, 'carp_diff'] = (ovin_df.iloc[:, 2:3].sum(axis=1) - lms_df.iloc[:, 2:3].sum(axis=1)) 

        zone.loc[:, 'train_ovin'] = ovin_df.iloc[:, 3:4].sum(axis=1)
        zone.loc[:, 'train_lms'] = lms_df.iloc[:, 3:4].sum(axis=1)
        zone.loc[:, 'train_diff'] = (- ovin_df.iloc[:, 3:4].sum(axis=1) + lms_df.iloc[:, 3:4].sum(axis=1)) / ovin_df.iloc[:, 3:4].sum(axis=1) * 100
        # zone.loc[:, 'train_diff'] = (ovin_df.iloc[:, 3:4].sum(axis=1) - lms_df.iloc[:, 3:4].sum(axis=1))


        zone.loc[:, 'btm_ovin'] = ovin_df.iloc[:, 4:5].sum(axis=1)
        zone.loc[:, 'btm_lms'] = lms_df.iloc[:, 4:6].sum(axis=1)
        zone.loc[:, 'btm_diff'] = (- ovin_df.iloc[:, 4:5].sum(axis=1) + lms_df.iloc[:, 4:6].sum(axis=1)) / ovin_df.iloc[:, 4:5].sum(axis=1) * 100
        # zone.loc[:, 'btm_diff'] = (ovin_df.iloc[:, 4:5].sum(axis=1) - lms_df.iloc[:, 4:6].sum(axis=1)) 

        zone.loc[:, 'bike_ovin'] = ovin_df.iloc[:, 5:6].sum(axis=1)
        zone.loc[:, 'bike_lms'] = lms_df.iloc[:, 6:7].sum(axis=1)
        zone.loc[:, 'bike_diff'] = (- ovin_df.iloc[:, 5:6].sum(axis=1) + lms_df.iloc[:, 6:7].sum(axis=1)) / ovin_df.iloc[:, 5:6].sum(axis=1) * 100
        # zone.loc[:, 'bike_diff'] = (ovin_df.iloc[:, 5:6].sum(axis=1) - lms_df.iloc[:, 6:7].sum(axis=1))


        zone.loc[:, 'walk_ovin'] = ovin_df.iloc[:, 6:7].sum(axis=1)
        zone.loc[:, 'walk_lms'] = lms_df.iloc[:, 7:8].sum(axis=1)
        zone.loc[:, 'walk_diff'] = (- ovin_df.iloc[:, 6:7].sum(axis=1) + lms_df.iloc[:, 7:8].sum(axis=1)) / ovin_df.iloc[:, 6:7].sum(axis=1) * 100
        # zone.loc[:, 'walk_diff'] = (ovin_df.iloc[:, 6:7].sum(axis=1) - lms_df.iloc[:, 7:8].sum(axis=1))

 
    
    zone[zone == np.inf] = np.nan

    return zone



In [ ]:
def plot_modal_split(zone, location, d='o', size=9,
                     labels=['Car', 'Public transport', 'Active modes']):

    f, ax = plt.subplots(int(size / 3), 3)
    f.set_figwidth(13)
    f.set_figheight(size + 3)

    # Get colormaps
    # cmap_du = plt.get_cmap('YlOrRd', 6)
    
    
    k = 0
    for i in range(int(size / 3)):
        zone_count = zone[zone.Count == 1]
        
        cmap_trip = plt.get_cmap('YlGn')
        min_val = np.min([zone_count.iloc[:, -size + k].min(), zone_count.iloc[:, -size + k + 1].min()])
        max_val = np.max([zone_count.iloc[:, -size + k].max(), zone_count.iloc[:, -size + k + 1].max()])
        min_val = 0
        max_val = np.round(max_val / 5) * 5

        for j in range(3):
            
            legend_label = f'Percentage of \n{labels[i]} trips'
            legend_label = f'Share of trips made by\n{labels[i]} [%]'
            if i < 2:
                legend_label = f'Share of trips made by\n{labels[i]}s [%]'

            if j == 0:
                source = 'OViN'
            elif j == 1:
                source = 'LMS'
            elif j == 2:
                source = '\nOViN - LMS difference'
                
                

                max_val = np.max([zone_count.iloc[:, -size + k].max(), np.abs(zone_count.iloc[:, -size + k].min())])
                min_val = -max_val

                max_val = np.round(max_val / 5) * 5
                min_val = -max_val
        

                max_val = 100
                min_val = -100
                cmap_trip = plt.get_cmap('RdBu_r')
                # cmap_trip = plt.get_cmap('RdBu')


                legend_label = f'%-points difference\nbetween OViN and LMS'
                legend_label = f'% difference \nbetween OViN and LMS'
            
            label = f'{labels[i]} : {source}'

            zone.plot(ax=ax[i, j], column=zone.columns[-size + k], vmin=min_val, vmax=max_val,
                       cmap=cmap_trip, legend=True,
                       legend_kwds={'label': legend_label})
            
            if len(zone[zone.Count == 0]) > 0:
                zone[zone.Count == 0].plot(ax=ax[i, j], facecolor='lightskyblue', edgecolor='black', linewidth=0.1)
            
            if j == 2:
                zone[zone.iloc[:, -size + k].isnull()].plot(ax=ax[i, j], facecolor='lightskyblue', edgecolor='black', linewidth=0.1)


            ax[i, j].set_title(label)

            if j != 2:
                ax[i, j].set_facecolor('lightskyblue')
            if j == 2:
                ax[i, j].set_facecolor('xkcd:light grey blue')
                ax[i, j].set_facecolor('lightskyblue')


            ax[i, j].set_xticks([])
            ax[i, j].set_yticks([])

            k += 1
    
    if d == 'o':
        dir = 'origin'
    else:
        dir = 'destination'

    if isinstance(location[0], int):
        location = ['Province'] 
    # f.suptitle(f'Modal split {" ".join(location)} for {dir} zone', fontsize=20)



In [ ]:
a = [1, 2]

isinstance(a[0], int)

In [ ]:
def execute(area, area_type='GEM_NAAM', size=9, 
            modes='c-btm-a',
            labels=['Car', 'Public transport', 'Active modes']):
    
    
    zone = select_zones(area, area_type=area_type)
    ovin_df, lms_df = select_ovin_lms(zone)
    
    zone = select_modes(zone, ovin_df, lms_df, modes=modes)
    plot_modal_split(zone, area, size=size, labels=labels)

    return zone


In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
area = ['Amsterdam']
area = ['Den Haag', 'Zoetermeer', 'Leiden', 'Delft']
# area = 

z = execute(area, size=18, modes='all',
        labels=['Car driver', 'Car passenger', 'Train',
                'BTM', '(e)-Bike', 'Walking'])

# DU

In [ ]:
f, ax = plt.subplots(1, 1)
f.set_figwidth(11)
f.set_figheight(5)

cmap = plt.get_cmap('YlOrRd', lut=6)

zones[zones.PROV.isin([10])].plot(ax=ax, column='deg_urba', cmap=cmap, vmin=1, vmax=6,
         legend=True, legend_kwds={"label": "Degree of urbanisation"})

ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor('lightskyblue')
ax.set_title(f'Degree of urbanisation for Zeeland');

In [ ]:
area = ['Den Haag', 'Zoetermeer', 'Leiden', 'Delft']
area = ['Amsterdam']


In [ ]:
f, ax = plt.subplots(1, 1)
f.set_figwidth(11)
f.set_figheight(5)

cmap = plt.get_cmap('YlOrRd', lut=6)

zones[zones.GEM_NAAM.isin(area)].plot(ax=ax, column='deg_urba', cmap=cmap, vmin=1, vmax=6,
         legend=True, legend_kwds={"label": "Degree of urbanisation"})

ax.set_xticks([])
ax.set_yticks([])
ax.set_facecolor('lightskyblue')
ax.set_title(f'Degree of urbanisation for Amsterdam');